# Our Task: Find similar company names

Now that we have completed our Python Primer, let's try to complete a real task, while at the same time trying to practice loops, iterations, and other Python functionality that we studied.

### Matching Company Names

Quite often, in our data, we have entries represented as strings that refer to the same entity but have different string representations (e.g., McDonald's vs McDonalds vs McDonald‎). We want to write code that will help in the task of finding such similar entries in our data.

Our task can be broken in the following tasks:
* **Step 1**: Read the data from a file into a list of strings in memory. We have two data sets under the `data` folder: The list of unique restaurant names from the NYC Restaurant Inspection dataset (`data/restaurant-names.txt`) and a list of bank names (`data/bank-names.txt`) We need to write Python code that will read these files and return a list of strings that are the company names.
* **Step 2**: We will then need to figure out how to compare two strings and find their similarity. For that, we will write a function that takes as input **two** strings, and returns back their similarities. We will explore multiple ways of writing that function, using various library functions.
* **Step 3**: We will need to write a function that takes as input a company name, and returns back a list of matching company names, together with their similarity. Ideally, we would like to also sort the list based on the similarity (highest similarity metrics on top). As part of our learning process, we will use the _list comprehension_ approach to create the list. We will also use tuples as the elements of the list, so that the list contain elements such as `[("McDonalds", 0.88), ("McDonald's", 0.81),....]`.
* **Step 4**: In the final step, we will just perform the similarity computation across all companies in the dataset.


### STEP 1: Read the list of names from a file and create a list of names

In [ ]:
# STEP 1: Read the list of names from a file and create a list of names
path = '/home/ubuntu/ipynb/NYU_Notes/2-Introduction_to_Python/'
filename = path + 'data/restaurant-names.txt'

### STEP 2: Implement the similarity function

### Computing the similarity between two strings

There are many ways that we can calculate the similarity between two strings. For our case, we will focus on a few similarity metrics that already have implementations in Python. 

##### Some commonly used similarity metrics

* Edit distance (aka Levenshtein distance) https://github.com/jamesturk/jellyfish
* N-gram distance (http://pythonhosted.org/ngram/tutorial.html#comparing-and-searching-strings)
* Sequence matching https://docs.python.org/2/library/difflib.html
http://stackoverflow.com/questions/17388213/find-the-similarity-percent-between-two-strings


In [8]:
# STEP 2: Now we implement our similarity function


In [ ]:
# This function takes as input two strings and then returns a number
# that is proportional to the similarity of the two strings.
# For consistency, we want this similarity number to be between 0 and 1,
# with 0 meaning no similarity, and 1 corresponding to identical strings

def computeSimilarity(str1, str2):
    # your code here
    pass
    


### STEP 3: Compute similarity of a company against a list of company names

We now create a function that accepts a company name and a list of companies, and computes their similarity. This part will get us to exercise our for-loops, and also illustrate how we can use lists and tuples.

**Sorting a list of tuples**:_This part is a little bit advanced for now, so I will just give the code below. (Solution taken from http://stackoverflow.com/questions/3121979/how-to-sort-list-tuple-of-lists-tuples). Here is a small example below, which we will reuse in our function:_

In [ ]:
data = [("Panos",0.5), ("Peter",0.6), ("Pan", 0.4)]
data.sort(key=lambda tupl: tupl[1], reverse=True)  # sorts in place, in descending order, based on the second element of the tuple
print data

In [ ]:
# STEP 3: We now create a function that accepts a company name
# and a list of companies, and computes their similarity
# We have a threshold parameter (by default set to be 3)
# that restricts the results to only the three most similar 
# string pairs 
def companySimilarity(companyName, companyList, threshold = 3):
    # your code here
    pass
      


In [ ]:
r = companySimilarity("STARBUCKS", company_names, 3)
r

### Step 4: Perform the similarity computation across all companies in the dataset.

In [ ]:
# STEP 4: We are almost done. We now just go through all the companies in the list
# and we call the companySimilarity function that computes the similar company names
# for all the companies in the list. We store the results in a dictionary, with the 
# key being the company name, and the value being a "list of tuples" with the 
# similar company names and the corresponding similarity value.
